# FAQ

#### * [pytraj is supposed to support more than 100 types of actions/analyses but I saw only a part of them in "pyca.common_actions", where is the rest?](#adict)
#### * [why do "pytraj" has something like "Action_Rmsd", "pytraj.common_actions.calc_rmsd", then "traj.calc_rmsd"](#whysomanysamemethods)
#### * [why are there so many dtype in calc_xyz (like, calc_dssp)? Why not using numpy only?](#notnumpy)
#### * [What is the different between "for frame in traj[1:11:2]" and "for frame in traj(1, 10, 2)", where "traj" is "TrajectoryIterator"?](#frameiter_vs_slicing)

<a name="adict"></a>
#### pytraj is supposed to support more than 100 types of actions/analyses but I saw only a part of them in pyca.common_actions, where is the rest?

Yes, `pytraj` supports all of `cpptraj` actions and most of analyses. You can create Action or Analysis object like 
```python
from pytraj.actions.Action_Rmsd import Action_Rmsd
```

In [1]:
from __future__ import print_function
from pytraj.actions.Action_Rmsd import Action_Rmsd
act = Action_Rmsd()
from pytraj import info
info(act)

this is what  you see when using `ipython` for python shell. This below text does appear in ipython-notebook since
it is C++ stdout

```
        [<name>] <mask> [<refmask>] [out filename] [nofit | norotate]
        [mass] [savematrices]
        [ first | reference | ref <name> | refindex <#> |
          reftraj <filename> [parm <parmname> | parmindex <#>] ]
        [perres perresout <filename> [perresavg <avgfile>]
         [range <resRange>] [refrange <refRange>]
         [perresmask <additional mask>] [perrescenter] [perresinvert]
  Calculate coordinate root-mean-squared deviation of atoms in <mask>
```

In [2]:
from pytraj.analyses.Analysis_Clustering import Analysis_Clustering
anal = Analysis_Clustering()
print (anal)

*** to make things easier, we created `adict` and `analdict` as a dictionary for actions and analyses***

In [3]:
from pytraj import adict
from pytraj import analdict
print (adict.keys().__len__()) # 80 actions
print (analdict.keys().__len__()) # 29 analyses

79
29


In [4]:
# let's check them out
print (adict.keys()[:10]) # only print first 10 actions
# create Action object
act_distance = adict['distance']
act_areapermol = adict['areapermol']

print (act_distance, act_areapermol)

['angle', 'areapermol', 'atomiccorr', 'atomicfluct', 'atommap', 'autoimage', 'average', 'bounds', 'box', 'center']
< Action_Distance object > < Action_AreaPerMol object >


In [5]:
# what am I supposed to do with those actions objects (I will show you so much shorter way later but I also want to show how
# `pytraj` work

# first, you creat DataSetList object and DataFileList object to hold internal data and to write files respectively
from pytraj import DataSetList, DataFileList

dslist = DataSetList()
dflist = DataFileList()

# but you need to load trajectory to `pytraj` too
import pytraj.io as io
traj = io.load("../tests/data/md1_prod.Tc5b.x", "../tests/data/Tc5b.top")
print (traj)

# let's make calculate
# the format is: act(command, traj, traj.top=Topology(), dslist=DataSetList(), dflist=DataFileList())

# let's make a dummy one to calculate distance for 3 atom maks: @CA @CB
# we need to have a Trajectory-like object (in thise case, we use `TrajReadOnly` for iterating frame by frame from disk
# we don't need to explicitly provide Topology since traj has this information
# we need to provide `dslist` and `dflist` object to hold and save data
# just like cpptraj, you need to specify `out` keyword to save to file (of course, you're not require to do so
act_distance("@CA @CB out output/test_act_distance.txt", traj, dslist=dslist, dflist=dflist)
print (dslist.size)
print (dflist)

print (dslist.tolist())

# explicitly call saving data
dflist.write_all_datafiles()

# check the result
!head -5 ./output/test_act_distance.txt

<pytraj.Trajectory with 10 frames: <Topology with 1 mols, 20 residues, 304 atoms, 310 bonds, non-PBC>>
           
1
[[1.2267204700233851, 1.0243572171177613, 0.7818476589515111, 1.3358773966222444, 1.106514609942215, 1.518443624705914, 1.281059038401526, 1.5526792445935809, 1.5580474960362545, 1.53805236103223]]
#Frame      Dis_00000
       1       1.2267
       2       1.0244
       3       0.7818
       4       1.3359


#### all of above steps can just be summarized in much simple way

In [6]:
import pytraj.common_actions as pyca

dslist = pyca.calc_distance(traj, "@CA @CB")
print (dslist.tolist())

[1.2267204700233851, 1.0243572171177613, 0.7818476589515111, 1.3358773966222444, 1.106514609942215, 1.518443624705914, 1.281059038401526, 1.5526792445935809, 1.5580474960362545, 1.53805236103223]


<a name="whysomanysamemethods"></a>
#### * why do `pytraj` has somethinge like `Action_Rmsd`, `pytraj.common_actions.calc_rmsd`, then `traj.calc_rmsd`

First of all, always look at:
```python
import pytraj.commmon_actions as pyca
```

In [7]:
import pytraj.common_actions as pyca
from pytraj.misc import get_atts
myatt = [d for d in get_atts(pyca) if d.startswith("calc")]
print (myatt)

['calc_COG', 'calc_COM', 'calc_angle', 'calc_atomicfluct', 'calc_center_of_geometry', 'calc_center_of_mass', 'calc_dih', 'calc_dihedral', 'calc_distance', 'calc_distrmsd', 'calc_dssp', 'calc_energies', 'calc_jcoupling', 'calc_matrix', 'calc_mindist', 'calc_molsurf', 'calc_multidihedral', 'calc_multivector', 'calc_pairwise_rmsd', 'calc_protein_score', 'calc_radgyr', 'calc_radial', 'calc_rdf', 'calc_rmsd', 'calc_score', 'calc_temperatures', 'calc_vector', 'calc_volmap', 'calc_volume', 'calc_watershell', 'calculate']


if there is nothing you need in `pyca`, look at
```python
from pytraj import adict
from pytraj import analdict

print (adict.keys())
print (analdict.keys())

# you can do something like
act = adict['volmap']
```

`pytraj` doc is still terriblly organized and miss a lot. We try to keep the same keyword for actions and analyses as in
`cpptraj`, so please read the Amber14 manual in [cpptraj section for detail](http://ambermd.org/doc12/Amber14.pdf). 
For example: you can search "volmap" keyword (page #549)
For quickly access the usage, use
```python
from pytraj import info
info(act) # act is either Action or Analysis object
# or
info('rmsd')
info('volmap')
```

<a name=notnumpy> </a>

#### why are there so many dtype in calc_xyz (like, calc_dssp)? Why not using numpy only?

#### * What is the different between "for frame in traj[1:11:2]" and "for frame in traj(1, 10, 2)", where "traj" is "TrajectoryIterator"?  <a name=frameiter_vs_slicing>

* Per result perspective, both of them are them same. Check below example

In [8]:
import pytraj.io as io
from pytraj.six_2 import zip
traj = io.load_sample_data("tz2")
print (traj)

for frame0, frame in zip(traj[1:11:2], traj(1, 10, 2)): 
    # we use `11` for the former because Python does not count the last index.
    print (frame0.rmsd(frame))  # two frames have the same coords
    

<pytraj.TrajectoryIterator with 10 frames: <Topology with 1692 mols, 1704 residues, 5293 atoms, 5300 bonds, PBC with box type = ortho>>
           
1.148761176495239e-06
1.4378647978776044e-06
1.292888048049322e-06
1.292888048049322e-06
0.0


* Per memory efficiency, traj(1, 10, 2) is better. This is shortcut of traj.frame_iter(start=1, stop=10, stride=2) (which is also similiar to cpptraj's command: `trajin my_trajectory.nc 1 10 2`. When calling `traj(1, 10, 2)`, only one single Frame is loaded into memory. 
```python
print (traj(1, 10, 2))
<generator object frame_iter at 0x2aaac8f7daf8>
```
In constrast, calling `traj[1:11:2]`, `pytraj` does the slicing first to create `Trajectory` object in memory, and then iterate frame
```python
print (traj[1:11:2])
<pytraj.Trajectory with 5 frames: <Topology with 1692 mols, 1704 residues, 5293 atoms, 5300 bonds, PBC with box type = ortho>>
```

In [9]:
%timeit for frame in traj(1, 10, 2): pass # faster and more memory efficient for `TrajectoryIterator`
%timeit for frame in traj[1:11:2]: pass

100 loops, best of 3: 3.25 ms per loop
100 loops, best of 3: 9.33 ms per loop


* if using `Trajectory` object (already in memory) rather than using `TrajectoryIterator`, using `frame_iter` is more efficient in speed

In [10]:
# convert to `TrajectoryIterator` (read only one single Frame from disk) to `Trajectory` (all frames are loaded into memory)
fa = traj[:]

%timeit for frame in fa(1, 10, 2): pass # about 20 times faster the later because we don't need to perform slicing.
%timeit for frame in fa[1:11:2]: pass

1000 loops, best of 3: 412 µs per loop
100 loops, best of 3: 5.98 ms per loop
